This is ML
 

In [ ]:
#import dependencies
import pandas as pd
import numpy as np
from datetime import date, datetime
import os, sys 
import io
import ee


In [ ]:
## set up pandas dataframe using set_options to warn when we are working on a copy instead of ooriginal dataframe
pd.set_option('mode.chained_assignment','warn')

In [ ]:
# dataframe will display without being truncated
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 768)

In [ ]:
# !pip install -U -q PyDrive

In [ ]:
#mounting the google drive to access the files
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# command shows file path
# ! ls

In [ ]:
n_path = "/content/drive/My Drive/data/naics-cat.csv"

In [ ]:
ndf = pd.read_csv( n_path, index_col=0, dtype={'cat':'string'}, na_values = ['no info', '.'], )
ndf.sample()

In [ ]:
# filepath
path = "/content/drive/My Drive/people_ppp.csv"




In [ ]:
#read the csv, create dataframe
# df = pd.read_csv(path,)
df = pd.read_csv( path, index_col=0, na_values = ['no info', '.'], )

In [ ]:
df.shape

In [ ]:
# df[weird]
for col in df.columns:
    weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis=1)
    if len(df[weird]) > 0:
        print(col)

In [ ]:
df.sample()

In [ ]:
df.loc[:,'NAICS Code'] = df.loc[:, 'NAICS Code'].astype(str)

In [ ]:
df.loc[:,'Zip'] = df.loc[:, 'Zip'].astype(str)

In [ ]:
df.loc[:,'Jobs Saved'] = df.loc[:, 'Jobs Saved'].astype(int)

In [ ]:
#somewhat forcing the date conversion because there must be some values that are not in date format 
df.loc[:,'Date'] =  pd.to_datetime(df.loc[:,'Date'], errors='coerce', )

In [ ]:
df.dtypes

In [ ]:
#check for nulls
# print(df.isna().sum())

In [ ]:
df.dropna(inplace=True)
df.sample()

In [ ]:
# print(df.isna().sum())

In [ ]:

for col in df.columns:
    weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis=1)
    if len(df[weird]) > 0:
        print(col)

In [ ]:
# weird = (df.applymap(type) != df.iloc[0].apply(type)).any(axis=1)
# df[weird]

In [ ]:
# #check for nulls
# print(df.isna().sum())


In [ ]:
#verified
df.shape

In [ ]:
#take the first two letters of the NAICS code column and create category column. 
#category column can be matched to named values. The longer code was impractical
df['NAICS Category'] = df['NAICS Code'].str[:2].astype(str)
df.info()

In [ ]:
df.sample()

In [ ]:
banks=df.loc[:,'Bank'].value_counts()

In [ ]:
banks.sample()

In [ ]:
# df['Bank'].where(df['Bank'].apply(lambda x:(x.value_counts()))>=500, "other")

In [ ]:
cond = df['Bank'].value_counts()
threshold = 500
df['Bank'] = np.where(df['Bank'].isin(cond.index[cond >= threshold ]), df['Bank'], 'miscellaneous')

In [ ]:
df['Bank'].value_counts()

In [ ]:
df.sample(3)

In [ ]:
df= df.groupby('NAICS Category').filter(lambda x: len(x) >= 5)


In [ ]:
# df.dropna(inplace=True)
df.head(1)

In [ ]:
df.shape

In [ ]:
# join n_path and df make merged df
mdf = pd.merge(left=df, right=ndf, left_on='NAICS Category', right_on='cat')
mdf.sample(12)







In [ ]:
merged_inner.shape

In [ ]:
mdf=mdf.drop(['cat', 'biz_count'], axis=1).copy()

In [ ]:
mdf.sample()

In [ ]:
mdf.shape
mdf.dtypes

In [ ]:
def to_csv(mdf, path):
    # Prepend dtypes to the top of df
    df2 = mdf.copy()
    df2.loc[-1] = df2.dtypes
    df2.index = df2.index + 1
    df2.sort_index(inplace=True)
    # Then save it to a csv
    df2.to_csv('p_cat_clean.csv', index=0)
    !cp p_cat_clean.csv "drive/My Drive/"
    df2.to_csv(path, index=False)

def read_csv(path):
    # Read types first line of csv
    path="/content/drive/My Drive/p_cat_clean.csv"
    dtypes = {key:value for (key,value) in pd.read_csv(path,    
              nrows=1).iloc[0].to_dict().items() if 'date' not in value}

    parse_dates = [key for (key,value) in pd.read_csv(path, 
                   nrows=1).iloc[0].to_dict().items() if 'date' in value]
    # Read the rest of the lines with the types from above
    return pd.read_csv(path, dtype=dtypes, parse_dates=parse_dates, skiprows=[1])

In [ ]:
# mdf.to_csv('p_cat_clean.csv', index=0)
# !cp p_cat_clean.csv "drive/My Drive/"

In [ ]:
# backup of clean dataframe, types will not be fully preserved so on import reassign, but easier with unconformity removed
scdf = pd.read_csv(mpath, dtype={'Jobs Saved':int, 'NAICS Code': str, 'NAICS Category': str, }, parse_dates=[10] )
scdf.sample()

In [ ]:
scdf.dtypes

In [ ]:
#convert the strings to integers in jobs saved
# df['Jobs Saved'] = pd.to_numeric(df['Jobs Saved'], errors="coerce")
# df['Jobs Saved'] = df['Jobs Saved'].astype(int)

In [ ]:
# df.to_csv('clean_ppp.csv', index=0)
# !cp clean_ppp.csv "drive/My Drive/"
